# Experiment 0: search for the best hyper parameters

### Using original vector as data representation. <br> 

## Libraries

In [1]:
from __future__ import print_function, division
from astropy.io          import fits
from astropy             import units as u
from astropy.coordinates import SkyCoord

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import OrderedDict
import math

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from torchsummary import summary
from torch.optim.lr_scheduler import LambdaLR, StepLR, MultiStepLR, ExponentialLR
import os
import argparse

import glob
import pickle
import CNN
import os
import sys

# Path

In [2]:
print(os.getcwd())

C:\Users\Administrator-1\Desktop\New folder (6)\HI\ML\Experiment_0_search_for_best_model


In [3]:
training_data_kok14_galfa = os.path.join(os.getcwd(),"data", "other", "training_data_kok14_galfa.pickle")

## Data preparation

In [4]:
data = pickle.load(open(training_data_kok14_galfa, 'rb'))

# training data
X_train = data['X_train']
Y_train = data['Y_train']
# Observed test data
X_test = data['X_test']
Y_test = data['Y_test']

In [5]:

x_train, x_test, y_train, y_test = train_test_split(X_train, 
                                                    Y_train, 
                                                    test_size=0.2, 
                                                    shuffle = True, 
                                                    random_state = 8)

x_train, x_val, y_train, y_val = train_test_split(x_train,
                                                  y_train, 
                                                  test_size=0.25, 
                                                  random_state= 8) # 0.25 x 0.8 = 0.2

In [6]:
print("x_train=",x_train.shape, ", y_train=", y_train.shape)
print("x_val=",x_val.shape, ", y_val=", y_val.shape)
print("x_test=",x_test.shape, ", y_test=", y_test.shape)

x_train= (23268, 414) , y_train= (23268, 2)
x_val= (7756, 414) , y_val= (7756, 2)
x_test= (7757, 414) , y_test= (7757, 2)


## Data loader

In [7]:
import data_loader
# initialize dataset
dataset_train = data_loader.spectra_loader(x_train, 
                                           y_train,
                                           transform=data_loader.ToTensor(),
                                           target_transform=data_loader.ToTensor(),
                                           pe=None)

dataset_val = data_loader.spectra_loader(x_val, 
                                         y_val,
                                         transform=data_loader.ToTensor(),
                                         target_transform=data_loader.ToTensor(),
                                         pe=None)

dataset_test = data_loader.spectra_loader(x_test,
                                          y_test,
                                          transform=data_loader.ToTensor(),
                                          target_transform=data_loader.ToTensor(),
                                          pe=None)

In [8]:
# initialize data loader
batch_size = 20 
train_loader = torch.utils.data.DataLoader(dataset = dataset_train,
                                           batch_size = batch_size,
                                           shuffle =True)

valid_loader = torch.utils.data.DataLoader(dataset = dataset_val,
                                           batch_size = batch_size,
                                           shuffle =False)

test_loader = torch.utils.data.DataLoader(dataset = dataset_test, 
                                          batch_size = batch_size,
                                          shuffle =False)

In [9]:
# test data loader
val = iter(test_loader)
A,B = next(val)
print(A.shape)
print(B.shape)

torch.Size([20, 1, 1, 414])
torch.Size([20, 2])


## Model Training and Evaluation 

In [74]:
# train 
def train(epoch):
    """
    Train the model.
    Parameters
    ----------
    epoch : int.
        the index of epoch.
    Returns
    -------
    train_loss/total: The mean MSE in an epoch. 
        
    """
    model.train()
    train_loss = 0
    total = 0
    for index, (inputs, values) in enumerate(train_loader):
        inputs = inputs.float()
        values = values.float()
        inputs, values = inputs.to(device), values.to(device)
        # forward
        outputs = model(inputs)
        loss = loss_function(outputs, values)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # print info
        train_loss = train_loss + (loss.item()*values.size(0)) 
        total += values.size(0)
    print('total loss = ', train_loss)
    return train_loss/total

In [75]:
def validation(epoch):
    """
    validate the model.
    Parameters
    ----------
    epoch : int.
        the index of epoch.

    Returns
    -------
    error: The mean MSE in validation set. 
        
    """
    model.eval()
    global best_err
    test_loss = 0
    total = 0
    with torch.no_grad():
        for index, (inputs, values) in enumerate(valid_loader):
            inputs = inputs.float()
            values = values.float()
            inputs, values = inputs.to(device), values.to(device)
            # forward
            outputs = model(inputs)
            loss = loss_function(outputs, values)
            test_loss = test_loss + (loss.item()*values.size(0))
            total += values.size(0)
    # Save checkpoint.
    error =  test_loss / total
    print(f"validation MSE in epoch {epoch}= ", error)
    if error < best_err:
        print('best_err:', error, 'Saving..')
        state = {'net': model.state_dict(),
                 'err': error,
                 'optimizer_state_dict': optimizer.state_dict(),
                 'epoch': epoch}
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/s2_no_pe.pth')
        best_err = error
    return error

In [76]:
def test(epoch):
    """
    test the model.
    Parameters
    ----------
    epoch : int.
        the index of epoch.
    Returns
    -------
    error: The mean MSE in test set. 
    """
    model.eval()
    global best_err
    test_loss = 0
    total = 0
    with torch.no_grad():
        for index, (inputs, values) in enumerate(test_loader):
            inputs = inputs.float()
            values = values.float()
            inputs, values = inputs.to(device), values.to(device)
            # forward 
            outputs = model(inputs)
            #outputs = outputs.view(outputs[0], 1, outputs[1])
            loss = loss_function(outputs, values)
            test_loss =test_loss+ (loss.item()* values.size(0))
            total += values.size(0)
        return test_loss/total

## Start 10 trail Training

In [77]:
def validate_model(num_output=2,
                   in_channels=1,
                   input_row = 1,
                   input_column=414,
                   num_layer = 10,
                   drop_out_rate=0.30,
                   lpe=False):
    global model
    model =  CNN.spectra_cnn(num_output=num_output,
                             in_channels=in_channels,
                             input_row = input_row,
                             input_column=input_column,
                             num_layer = num_layer,
                             drop_out_rate=drop_out_rate,
                             lpe=lpe)
    num_step = len(X_train)/batch_size
    loss_function = nn.MSELoss()
    best_err = 100000
    checkpoint = torch.load('./checkpoint/s2_no_pe.pth',map_location=torch.device('cuda:0'))
    model.load_state_dict(checkpoint['net'])
    model.to(device)
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    validation_error = validation(epoch)
    test_error = test(epoch)
    return validation_error, test_error

In [78]:
def trail(num_epoch,
          num_output=2,
          in_channels=1,
          input_row = 1,
          input_column=414,
          num_layer = 10,
          drop_out_rate=0.30,
          lpe=False):
    
    global vali
    global testing
    global scheduler
    global model
    global learning_rate
    lr = learning_rate
    #define savor
    global train_trails
    global validate_trails
    global test_trails
    
    for epoch in range(num_epoch):
        train_err = train(epoch)
        vali_err = validation(epoch)
        scheduler.step()
        test_err = test(epoch)
        print('train_err=', train_err)
        # save data
        train_trails.append(train_err)
        validate_trails.append(vali_err)
        test_trails.append(test_err)
    # final test
    validation_error,test_error = validate_model(num_output=num_output,
                                                 in_channels=in_channels,
                                                 input_row = input_row,
                                                 input_column=input_column,
                                                 num_layer = num_layer,
                                                 drop_out_rate=drop_out_rate,
                                                 lpe=lpe)
    print("test err=",test_error)
    vali.append(validation_error)
    testing.append(test_error)

In [79]:
# start 10 trails.
def trail_10(num_trial,
             num_epoch, 
             #
             in_channels,
             input_row,
             input_column,
             num_layer,
             num_output=2,
             drop_out_rate=0.3,
             lpe=False):
    
    global learning_rate
    global vali
    global testing
    global train_trails
    global validate_trails
    global test_trails
    
    for i in range (0, num_trial):
        print(f'start the {i}th trial:')
        # model initialization
        lr=learning_rate
    
        model =  CNN.spectra_cnn(num_output=num_output,
                             in_channels=in_channels,
                             input_row = input_row,
                             input_column=input_column,
                             num_layer = num_layer,
                             drop_out_rate=drop_out_rate,
                             lpe=lpe)
    
        num_step = len(X_train)/batch_size
        loss_function = nn.MSELoss()
        best_err = 100000
        optimizer = torch.optim.SGD(model.parameters(), lr)
        scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer=optimizer, 
                                                     milestones = [65], 
                                                     gamma=0.1, last_epoch=-1, 
                                                     verbose=False)
        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        model.to(device)
        trail(num_epoch = num_epoch,
              input_row = input_row,
              in_channels = in_channels,
              input_column=input_column,
              num_layer = num_layer,
              drop_out_rate=drop_out_rate,
              lpe=lpe,
              num_output=num_output
             )

## Results display methods

In [80]:
def validation_performance(vali):
    x = vali
    plt.figure(figsize=(7,5))
    plt.title('validation performance')
    plt.plot(x, label='validation MSE',marker='o')
    plt.plot([], [], ' ', label=f"mean={round(np.mean(x),6)}")
    plt.plot([], [], ' ', label=f"std={round(np.std(x),6)}")
    plt.xlabel('Trail')
    plt.ylabel('MSE')
    plt.legend()
    plt.grid()
    plt.show()
    print("validation mean=",np.mean(x))
    print("validation std=",np.std(x))

In [81]:
def test_performance(testing):
    x = testing
    plt.figure(figsize=(7,5))
    plt.title('testing performance')
    plt.plot(x, label='testing MSE',marker='o')
    plt.plot([], [], ' ', label=f"mean={round(np.mean(x),6)}")
    plt.plot([], [], ' ', label=f"std={round(np.std(x),6)}")
    plt.xlabel('Trail')
    plt.ylabel('MSE')
    plt.legend()
    plt.grid()
    plt.show()
    print("testing mean=",np.mean(x))
    print("testing std=",np.std(x))

In [82]:
def vali_testing_performance_show(vali, testing):
    x1 = vali
    x2 = testing
    plt.figure(figsize=(7,5))
    plt.title('Two in one')
    plt.plot(x1, label='vaidation_MSE',marker='o')
    plt.plot(x2, label='tesing_MSE',marker='o')
    plt.xlabel('Trail')
    plt.ylabel('MSE')
    plt.legend()
    plt.grid()
    plt.show()

## Save csv methdos

In [83]:
def save_csv(vali, testing, file_name_1, file_name_2):
    x1 = np.array(vali).reshape(1, -1)
    x2 = np.array(testing).reshape(1, -1)
    two = np.vstack((x1,x2))
    df = pd.DataFrame(two).T
    df.to_csv(os.path.join(os.getcwd(),"training_process") + "\\" + file_name_1)
    #
    x3 = np.array(train_trails).reshape(1, -1)
    x4 = np.array(validate_trails).reshape(1, -1)
    x5 = np.array(test_trails).reshape(1, -1)
    three = np.vstack((x3,x4,x5))
    df = pd.DataFrame(three).T
    df.to_csv(os.path.join(os.getcwd(),"training_process") + "\\"+ file_name_2)

## Start training for layer number = [2, 4, 6, 8, 10, 12, 14, 16]

In [84]:
num_layer = [2, 4, 6, 8, 10, 12, 14, 16]

In [85]:
for i in num_layer:
    learning_rate = 0.005
    # the savier of best validation performance and coresponding testing performance.
    vali = []
    testing = []
    # saver for each trials
    train_trails = []
    validate_trails =[]
    test_trails = []
    print("num_layer=", i)
    trail_10(num_trial = 10,
             num_epoch = 80, 
             #
             in_channels = 1,
             input_row=1,
             input_column=414,
             num_layer = i,
             num_output=2,
             drop_out_rate=0.3,
             lpe=False)
    validation_performance(vali)
    test_performance(testing)
    vali_testing_performance_show(vali, testing)
    file_name_1 = f'num_layer_{i}.csv'
    file_name_1 = f'num_layer_{i}_process.csv'
    save_csv(vali, testing, file_name_1, file_name_2)

num_layer= 2
start the 0th trial:


KeyboardInterrupt: 